# Parser pour récupérer des données de site www.jeuxdemots.org

### improt 

In [1]:
import requests
import pandas as pd 

### Chorisir son mot 

In [2]:
PATH_REQUEST = "data/requests/"

# Arborescence du sysème de fichier 
PATH_REQUEST = "data/requests/"
PATH_DEF  = "data/def/"
PATH_E    = "data/e/"
PATH_NT   = "data/nt/"
PATH_RE   = "data/re/"
PATH_RS   = "data/rs/"
PATH_RT   = "data/rt/"

# Exrension des fichiers 
EXT_REQUEST = ".txt"
EXT_DEF  = "_def.csv"
EXT_E    = "_e.csv"
EXT_NT   = "_nt.csv"
EXT_RE   = "_re.csv"
EXT_RS   = "_rs.csv"
EXT_RT   = "_rt.csv"

mot = "chaud"

### Télécharger la page web contenant les données : 

In [3]:
def request(mot): 
    url = 'https://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=' + mot + '&rel='
    headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
    r = requests.post(url, headers=headers)

    # Enregistrer la requetes dans un fichier text
    f = open(PATH_REQUEST + mot + EXT_REQUEST, "w")
    f.write(r.text)
    f.close()

    f = open(PATH_REQUEST + mot + EXT_REQUEST, "r")
    #f = open(PATH_REQUEST + "goodPage.html", "r")
    #f = open(PATH_REQUEST + "badPage.html", "r")
    lines = f.readlines()
    f.close()

    return "<br>//&nbsp; &nbsp; &nbsp; WARNING TROP GROS.<br>TOWARD CACHE<br>\n" not in lines



### Parser les données

In [4]:
def download(mot): 
    i = 0

    while not request(mot) and i < 5: 
        print("Pb download")
        i += 1

In [5]:
download(mot)

In [6]:
f = open(PATH_REQUEST + mot + EXT_REQUEST, "r")
lines = f.readlines()
f.close()


code = [lines.index('<CODE>\n'), lines.index('</CODE>\n') +1]
print(code)

index = {}

for i in range(code[0], code[1]):
    if lines[i] == "<def>\n":
        index["def"] = i

    if lines[i] == "// les types de noeuds (Nodes Types) : nt;ntid;'ntname'\n":
        index["nt"] = i

    if lines[i] == "// les noeuds/termes (Entries) : e;eid;'name';type;w;'formated name' \n":
        index["e"] = i

    if lines[i] == "// les types de relations (Relation Types) : rt;rtid;'trname';'trgpname';'rthelp' \n":
        index["rt"] = i

    if lines[i] == "// les relations sortantes : r;rid;node1;node2;type;w \n":
        index["rs"] = i

    if lines[i] == "// les relations entrantes : r;rid;node1;node2;type;w \n":
        index["re"] = i 

    if lines[i] == "// END\n":
        index["end"] = i 

index


[138, 7412]


{'def': 141,
 'nt': 147,
 'e': 163,
 'rt': 2672,
 'rs': 2730,
 're': 4845,
 'end': 7410}

In [7]:
m_def   = open(PATH_DEF + mot + EXT_DEF, "w")
m_nt    = open(PATH_NT  + mot + EXT_NT,  "w")
m_e     = open(PATH_E   + mot + EXT_E,   "w")
m_rt    = open(PATH_RT  + mot + EXT_RT,  "w")
m_rs    = open(PATH_RS  + mot + EXT_RS,  "w")
m_re    = open(PATH_RE  + mot + EXT_RE,  "w")

# Head
m_nt.write("nt;ntid;ntname\n")
m_e.write("e;eid;name;type;w;help\n")
m_rt.write("rt;rtid;trname;trgpname;rthelp\n")
m_rs.write("r;rid;node1;node2;type;w\n")
m_re.write("r;rid;node1;node2;type;w\n")

for i in range(index['def'], index['nt']):
    m_def.write(lines[i])

for i in range(index['nt']+2, index['e']):
    m_nt.write(lines[i].replace("'", ""))

for i in range(index['e']+2, index['rt']):
    if lines[i].count(";") <= 5:
        m_e.write(lines[i].replace("'", ""))

for i in range(index['rt']+2, index['rs']):
    m_rt.write(lines[i].replace(" ; ", " ").replace("'", ""))

if 're' in index and 'rs' in index: 
    for i in range(index['rs']+2, index['re']):
        m_rs.write(lines[i])

    for i in range(index['re']+2, index['end']):
        m_re.write(lines[i])
elif 'rs' in index: 
    for i in range(index['rs']+2, index['end']):
        m_rs.write(lines[i])
elif 're' in index: 
    for i in range(index['re']+2, index['end']):
        m_re.write(lines[i])


m_def.close()
m_nt.close()
m_e.close()
m_rt.close()
m_rs.close()
m_re.close()

In [27]:
E  = pd.read_csv(PATH_E  + mot + EXT_E,  sep=";")
NT = pd.read_csv(PATH_NT + mot + EXT_NT, sep=";")
RE = pd.read_csv(PATH_RE + mot + EXT_RE, sep=";")
RS = pd.read_csv(PATH_RS + mot + EXT_RS, sep=";")
RT = pd.read_csv(PATH_RT + mot + EXT_RT, sep=";")
R  = pd.concat([RE, RS])

In [28]:
R

,r,rid,node1,node2,type,w
0,r,61298259,114327,19150,0,-82
1,r,60569902,74846,19150,0,-36
2,r,60571678,95422,19150,0,-34
3,r,60564180,139779,19150,0,-32
4,r,80440940,16747,19150,0,-32
...,...,...,...,...,...,...
2107,r,284258069,19150,1960596,999,30
2108,r,473638927,19150,1218553,2000,10
2109,r,333902933,19150,1218554,2000,15
2110,r,307601219,19150,439630,2000,25


### One Cell

In [8]:
badLine = "e;7109740;en:glossina <genus>;1;0;en:glossina &lt;genus&gt;"
goodLine = "e;9072515;en:elevator;1;0"
print(goodLine.count(";")) 
print(badLine.count(";")) 

#line.replace(";", "|", 5)

4
7


In [30]:
f = open(PATH_REQUEST + "goodPage.html", "r")
lines = f.readlines()

if "<br>//&nbsp; &nbsp; &nbsp; WARNING TROP GROS.<br>TOWARD CACHE<br>\n" in lines : 
    print("La page n'est pas bonne !")
else : 
    print ("La page est bonne !")


La page est bonne !
